In [ ]:
import yaml
no2_price_scenario_mappings = []
for attr_name in ["buy_price", "sale_price"]:
    for price_offset in range(-200, 200+1, 20):
        no2_price_scenario_mappings.append({
            "external_id": f"no2_{attr_name}_offset_{price_offset}",
            "object_type": "market",
            "object_name": "Dayahead",
            "attribute_name": attr_name,
            "time_series": f"[external_id]907677",
            "transformations": [{
                    "AddFromOffset": {
                        "parameters": {
                            "relative_datapoints": [
                                {"offset_minute": 0.0, "offset_value": price_offset},
                                {"offset_minute": 1440.0, "offset_value": 0.0},
                            ]
                        }
                    }
                },
            ],
        })

print(yaml.safe_dump(no2_price_scenario_mappings, sort_keys=False))

In [ ]:
with open("../tests/data/demo/cogshop/time_series_mappings_v2.yaml", "r") as f:
    old_mappings = yaml.safe_load(f)[0]["rows"]

fornebu_base_mappings = []
for mapping in old_mappings:
    fornebu_base_mappings.append({
        "external_id": f"base_mapping_fornebu_{mapping['object_type']}_{mapping['object_name']}_{mapping['attribute_name']}".lower(),
        "object_type": mapping["object_type"],
        "object_name": mapping["object_name"],
        "attribute_name": mapping["attribute_name"],
        "time_series": f"[external_id]{mapping['time_series_external_id']}" if mapping.get("time_series_external_id") else None,
        "transformations": mapping.get("transformations", []),
        "retrieve": mapping.get("retrieve", "START" if "attribute_name" in ["start_head", "start_vol"] else "RANGE"),
        "aggregation": mapping.get("aggregation", "MEAN"),
    })

print(yaml.safe_dump(fornebu_base_mappings, sort_keys=False))

In [ ]:
plants_with_schedule = list({mapping["object_name"] for mapping in fornebu_base_mappings if mapping["object_type"] == "plant" and mapping["attribute_name"] == "production_schedule"})
for plant in plants_with_schedule:
    fornebu_base_mappings.append({
        "external_id": f"base_mapping_fornebu_plant_{plant}_production_schedule_flag".lower(),
        "object_type": "plant",
        "object_name": plant,
        "attribute_name": "production_schedule_flag",
        "time_series": None,
        "transformations": [{
            "StaticValues":{
              "parameters": {
                  "relative_datapoints": [{
                  "offset_minute": 0.0,
                  "offset_value": 0
                  }]}}}],
    })

## Print all mappings
To be pasted into `/tests/data/demo/v1/fornebu/shop_attribute_mappings.yaml`

In [ ]:
mappings = fornebu_base_mappings + no2_price_scenario_mappings
print(yaml.safe_dump(mappings, sort_keys=False))

## Print base mapping external IDs for Fornebu
To be pasted into `/tests/data/demo/v1/fornebu/shop_model.yaml`

In [ ]:
for bm in fornebu_base_mappings:
    print("-", f"\"[external_id]{bm['external_id']}\"")

# Day-ahead bid configs

In [ ]:
plants = plants_with_schedule
for plant in plants:
    for add_steps in [True, False]:
        for number_of_scenarios_in_scenario_set in ["2", "3"]:
            print(f"- name: {plant} {number_of_scenarios_in_scenario_set}{' step' if add_steps else ''}")
            print(f"  method: multi scenario {number_of_scenarios_in_scenario_set}")
            print(f"  power_asset: \"[external_id]plant_water_value_based_{plant.lower()}\"")
            print(f"  add_steps: {add_steps}")
            print(f"  scenario_set: \"[name|type:ShopScenarioSet]{number_of_scenarios_in_scenario_set} scenarios\"")
  

In [ ]:
number_of_scenarios_in_scenario_set = "2"
for plant in plants:
    print(f'    - "[external_id]shop_based_partial_bid_configuration_{plant.lower()}_{number_of_scenarios_in_scenario_set}"')